# 군집분석
  - 비지도 학습 : 정답이 없다
  - 지도 학습 : 회귀, 분류

In [ ]:
# 소비자의 군집분석(타겟 마케팅)

# k-평균 알고리즘: kluster 군집의 개수
  - 임의의 K 개수만큼 점을 찍고 각 점과 데이터사이의 거리를계산(유클리디안 거리의 제곱)한 다음 평균지점으로 이동을 계속 하면서 더 이상 이동 할 수 없을때 까지 이동한다.
  - 단점 : 클러스터의 개수를 직접 지정
    - 적합한 개수는 : 엘보우 기법으로 확인한다
  - 엘보우기법
    - 왜곡 : 중심점과 데이터의 거리의 제곱합
    - K 개수를 늘리면서 그래프를 그려보면 꺽이는 지점
  - 실루엣 분석 : 시각화
    - 클러스터내에 있는 데이터가 얼마나 모여있는지 측정하는 도구
    - 실루엣 계수 : -1 ~ 1사이의 값을 가진다 1에 가까울 수록 좋은 군집화
  


In [ ]:
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
online_retail = fetch_ucirepo(id=352)

# data (as pandas dataframes)
X = online_retail.data.features
y = online_retail.data.targets

In [ ]:
X.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Description  540455 non-null  object 
 1   Quantity     541909 non-null  int64  
 2   InvoiceDate  541909 non-null  object 
 3   UnitPrice    541909 non-null  float64
 4   CustomerID   406829 non-null  float64
 5   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 24.8+ MB


In [ ]:
# 이 데이터의 핵심은 고객에 대한 마케팅... 고객중요
# 고객 번호가 없는데이터는?

In [ ]:
idx = X.isna().apply(lambda x: x.any(), axis=1)
X[idx]

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...
541536,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


In [ ]:
X[X['Description'].isna()]

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...
535322,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


In [ ]:
# Description null인 데이터는 제거한다
X = X.dropna(subset=['Description'])

In [ ]:
X.reset_index(drop=True, inplace=True)

In [ ]:
# 중복데이터 제거
X.drop_duplicates(inplace=True)

<ipython-input-36-44cff28894ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop_duplicates(inplace=True)


In [ ]:
X.reset_index(drop=True, inplace=True)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534532 entries, 0 to 534531
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Description  534532 non-null  object 
 1   Quantity     534532 non-null  int64  
 2   InvoiceDate  534532 non-null  object 
 3   UnitPrice    534532 non-null  float64
 4   CustomerID   401528 non-null  float64
 5   Country      534532 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 24.5+ MB


In [ ]:
X.iloc[1400:1427]

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1400,JUMBO STORAGE BAG SUKI,40,12/1/2010 14:11,1.95,16456.0,United Kingdom
1401,JUMBO BAG TOYS,20,12/1/2010 14:11,1.95,16456.0,United Kingdom
1402,JUMBO BAG OWLS,30,12/1/2010 14:11,1.95,16456.0,United Kingdom
1403,WHITE HANGING HEART T-LIGHT HOLDER,32,12/1/2010 14:11,2.95,16456.0,United Kingdom
1404,HAND WARMER RED RETROSPOT,-1,12/1/2010 14:30,2.10,17841.0,United Kingdom
1405,CHARLOTTE BAG SUKI DESIGN,-2,12/1/2010 14:30,0.85,17841.0,United Kingdom
1932,3 HOOK HANGER MAGIC GARDEN,-4,12/1/2010 14:33,1.95,12472.0,Germany
1933,5 HOOK HANGER MAGIC TOADSTOOL,-5,12/1/2010 14:33,1.65,12472.0,Germany
1934,SET/5 RED RETROSPOT LID GLASS BOWLS,-1,12/1/2010 14:33,2.95,12472.0,Germany
1935,SET OF SALT AND PEPPER TOADSTOOLS,-7,12/1/2010 14:33,1.25,12472.0,Germany


In [ ]:
# CustomerID가 null인 항목 제외

# idx = X['CustomerID'].isna()
# X = X[~idx]
X.dropna(subset=['CustomerID'], inplace=True)

<ipython-input-48-b3e99bcc33c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(subset=['CustomerID'], inplace=True)


In [ ]:
X.shape

(401528, 6)

In [ ]:
# Quantity 값이 양수인 데이터 추출
idx = X['Quantity'] > 0
X = X[idx]
X.shape

(392657, 6)